# Benchmark

In [ ]:
DIAG = "Gracious Diagnosis"

import os
import paramiko
import seaborn as sns
import matplotlib.pyplot as plt
from paramiko import Ed25519Key
from datetime import datetime
import shutil
from cmcrameri import cm
import palettable
from matplotlib.colors import ListedColormap

prettify_model_name = {
    "Llama-2-70B-chat-GPTQ": "Llama 2 Chat",
    "Llama2-70B-OASST-SFT-v10-GPTQ": "OASST",
    "WizardLM-70B-V1.0-GPTQ": "WizardLM",
    "axiong_PMC_LLaMA_13B": "PMC Llama",
    "ClinicalCamel-70B-GPTQ": "Clinical Camel",
    "Meditron-70B-GPTQ": "Meditron",
    "MIMIC Doctors": "MIMIC Doctors"
}

bat_4_cmap = ListedColormap(palettable.scientific.sequential.Batlow_4.mpl_colors)
davos_5_cmap = ListedColormap(palettable.scientific.sequential.Hawaii_5.mpl_colors)

Bilbao_4_cmap = ListedColormap(palettable.scientific.sequential.Bilbao_4.mpl_colors)
Devon_4_cmap = ListedColormap(palettable.scientific.sequential.Devon_4.mpl_colors)
Tokyo_4_cmap = ListedColormap(palettable.scientific.sequential.Tokyo_4.mpl_colors)

color_map = {
    "Appendicitis" : davos_5_cmap.colors[0],
    "Cholecystitis" : Devon_4_cmap.colors[1],
    "Diverticulitis" : davos_5_cmap.colors[2],
    "Pancreatitis" : davos_5_cmap.colors[3],
    "Llama 2 Chat": bat_4_cmap.colors[0],
    "OASST": bat_4_cmap.colors[1],
    "WizardLM": bat_4_cmap.colors[2],
    "Clinical Camel" : Tokyo_4_cmap.colors[1],
    "Meditron" : Tokyo_4_cmap.colors[2],
    "MIMIC Doctors": Bilbao_4_cmap.colors[-1],
    "Doctors": Bilbao_4_cmap.colors[-2],
    "Mean" : davos_5_cmap.colors[4]
}


from utils.logging import read_from_pickle_file

from os.path import join
from dataset.utils import load_hadm_from_file

from evaluators.appendicitis_evaluator import AppendicitisEvaluator
from evaluators.cholecystitis_evaluator import CholecystitisEvaluator
from evaluators.diverticulitis_evaluator import DiverticulitisEvaluator
from evaluators.pancreatitis_evaluator import PancreatitisEvaluator
from run import load_evaluator
from utils.nlp import latex_escape

# Check new evaluation strategy
def load_evaluator(pathology):
    # Load desired evaluator
    if pathology == "appendicitis":
        evaluator = AppendicitisEvaluator()
    elif pathology == "cholecystitis":
        evaluator = CholecystitisEvaluator()
    elif pathology == "diverticulitis":
        evaluator = DiverticulitisEvaluator()
    elif pathology == "pancreatitis":
        evaluator = PancreatitisEvaluator()
    else:
        raise NotImplementedError
    return evaluator

import glob

import numpy as np
import pickle

def calculate_average(evals, field, pathology):
  average = 0
  for patient in evals.keys():
    if field not in evals[patient]['scores']:
        print(f"{field} not in {patient}")
    average += evals[patient]['scores'][field]

  average /= len(evals)
  #print(f'{pathology}: {average:0.02} (n={len(evals)})'.rjust(30))
  return average, len(evals)

def calculate_percentages(evals, field):
    max_field = field
    if "Late" in field:
        max_field = max_field.replace("Late ", "")
    max_field = max_field.replace(" Percentage", "")
    for patient in evals.keys():
        evals[patient]['scores'][field] = evals[patient]['scores'][field[:-len(" Percentage")]] / evals[patient]['max_scores'][max_field]
    return evals

def count_unnecessary(evals, field):
    for patient in evals.keys():
        evals[patient]['scores'][field] = len(evals[patient]['answers'][field])
    return evals


def print_results(difficulty, all_evals):
    id_difficulty = pickle.load(open('id_difficulty.pkl', 'rb'))
    avg_scores = {}
    avg_samples = {}
    print(f'@@@ {difficulty} @@@'.center(30))
    print()
    if difficulty in ['easy', 'secondary', 'hard']:
        all_evals_diff = {}
        for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']:
            all_evals_diff[patho] = {}
            for _id in all_evals[patho].keys():
                if _id in id_difficulty[patho][difficulty]:
                    all_evals_diff[patho][_id] = all_evals[patho][_id]
    else:
        all_evals_diff = all_evals
    for field in ['Diagnosis', 'Custom Parsings', 'Rounds', 'Physical Examination', 'Unnecessary Laboratory Tests', 'Unnecessary Imaging']:
        avg_scores[field] = {}
        avg_samples[field] = {}
        print(f'### {field} ###'.center(30))
        for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']:
            if field in ['Unnecessary Laboratory Tests', 'Unnecessary Imaging']:
                all_evals_diff[patho] = count_unnecessary(all_evals_diff[patho], field)

            avg, n = calculate_average(all_evals_diff[patho], field, patho)
            print(f'{patho}: {avg:0.02} (n={n})'.rjust(30))

            avg_scores[field][patho] = avg
            avg_samples[field][patho] = n
        print(f'AVERAGE: {np.mean(list(avg_scores[field].values())):0.2} (n={round(np.mean(list(avg_samples[field].values())))})'.rjust(30))
        print()

# Check new evaluation strategy
def load_evaluator(pathology):
    # Load desired evaluator
    if pathology == "appendicitis":
        evaluator = AppendicitisEvaluator()
    elif pathology == "cholecystitis":
        evaluator = CholecystitisEvaluator()
    elif pathology == "diverticulitis":
        evaluator = DiverticulitisEvaluator()
    elif pathology == "pancreatitis":
        evaluator = PancreatitisEvaluator()
    else:
        raise NotImplementedError
    return evaluator

import pandas as pd

def generate_latex_tables(model_scores, experiment_name):
    model_dicts = list(model_scores.values())
    model_names = list(model_scores.keys())
    diseases = ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']
    categories = list(model_dicts[0].keys())

    model_names = [prettify_model_name[name] for name in model_names]
    category_names  = [category.replace(" ", " \\\\ ") for category in categories]

    # for mean performance
    data = []
    for model_dict, model_name in zip(model_dicts, model_names):
        row = [model_name] + [sum(model_dict[category].values())/len(diseases) for category in categories]
        data.append(row)
    df = pd.DataFrame(data, columns=["Model"] + [f"\\thead{{{category}}}" for category in category_names])
    styler = df.style
    styler.format({f"\\thead{{{category}}}": "{:.2f}" for category in category_names}).hide(level=0, axis=0)
    print("\\begin{table}[ht]")
    print("\\begin{adjustwidth}{-1in}{-1in}")
    print("\\centering")
    if experiment_name:
        experiment_name = f" - {experiment_name}"
    print(f"\\caption{{Mean model performance{experiment_name}}}")
    print(styler.to_latex(column_format='l' + 'c'*len(categories), hrules=True))
    print("\\end{adjustwidth}")
    print("\\end{table}\n")

    for disease in diseases:
        data = []
        for model_dict, model_name in zip(model_dicts, model_names):
            row = [model_name] + [model_dict[category][disease] for category in categories]
            data.append(row)
        df = pd.DataFrame(data, columns=["Model"] + [f"\\thead{{{category}}}" for category in category_names])
        styler = df.style
        styler.format({f"\\thead{{{category}}}": "{:.2f}" for category in category_names}).hide(level=0, axis=0)
        print("\\begin{table}[ht]")
        print("\\begin{adjustwidth}{-1in}{-1in}")
        print("\\centering")

        print(f"\\caption{{Model performance on {disease.capitalize()}{experiment_name}}}")
        print(styler.to_latex(column_format='l' + 'c'*len(categories), hrules=True))
        print("\\end{adjustwidth}")
        print("\\end{table}\n")

def generate_latex_tables_full_info(model_scores, experiment_name, bold_max=True, underline_second_max=True):
    diseases = ['Mean', 'Appendicitis', 'Cholecystitis', 'Diverticulitis', 'Pancreatitis']
    models = list(model_scores.keys())

    data = []
    for model_name in models:
        row = []
        for disease in diseases:
            row.append(model_scores[model_name][disease])
        data.append(row)

    df = pd.DataFrame(data, columns=diseases, index=models)

    if bold_max or underline_second_max:
        def format_number(number, column, max_number, second_max_number):
            if bold_max and number == max_number:
                return "\\textbf{%.2f}" % number
            elif underline_second_max and number == second_max_number:
                return "\\underline{%.2f}" % number
            else:
                return "%.2f" % number

        for column in df.columns:
            max_number = df[column].max()
            second_max_number = df[df[column] != max_number][column].max() if underline_second_max else None
            df[column] = df[column].apply(lambda num: format_number(num, column, max_number, second_max_number))

    latex_content = df.to_latex(escape=False, multirow=True, column_format="|c"* (len(diseases) + 1) + "|")

    for disease in diseases:
        latex_content = latex_content.replace(f'\\multicolumn{{2}}{{r}}{{{disease}}}', f'\\multicolumn{{2}}{{|c|}}{{\\textbf{{{disease}}}}}')

    print("\\begin{table}[ht]")
    print("\\centering")
    if experiment_name:
        experiment_name = f" - {latex_escape(experiment_name)}"
    print(f"\\caption{{Diagnostic Accuracy (\%) with Full Information{experiment_name}}}")
    print(latex_content)
    print("\\end{table}\n")

def load_scores(experiments, difficulty="first_diag", fields=['Diagnosis'], models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ"], pathos = ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']):
    id_difficulty = pickle.load(open('id_difficulty.pkl', 'rb'))

    experiment_results = {}
    experiment_evals = {}
    experiment_scores = {}
    for experiment in experiments:
        model_results = {}
        model_evals = {}
        model_scores = {}
        for model in models:
            m_results = pickle.load(open(f'logs/SOTA/{experiment}/{model}_results.pkl', 'rb'))
            m_evals = pickle.load(open(f'logs/SOTA/{experiment}/{model}_evals.pkl', 'rb'))
            model_results[model] = m_results
            model_evals[model] = m_evals

        for model in models:
            all_evals = {}
            all_results = {}
            for patho in pathos:
                all_evals[patho] = model_evals[model][patho]
                all_results[patho] = model_results[model][patho]
                
                # Subset to only desired ids
                selected_evals = {}
                selected_results = {}
                for _id in id_difficulty[patho][difficulty]:
                    if _id not in all_evals[patho]:
                        # Manually tested and all were correct
                        if _id == 21285450:
                            selected_evals[_id] = {'scores': {'Diagnosis': 1.0, "Gracious Diagnosis": 1.0, "Action Parsing": 0, "Treatment Parsing": 0, "Diagnosis Parsing": 0, 'Invalid Tools': 0}}
                        else:
                            print(f"For experiment {experiment} and model {model}, {_id} not in {patho}")
                        continue
                    selected_evals[_id] = all_evals[patho][_id]
                    selected_results[_id] = all_results[patho][_id]
                all_evals[patho] = selected_evals
                all_results[patho] = selected_results
            
            model_evals[model] = all_evals
            model_results[model] = all_results
            avg_scores = {}
            avg_samples = {}

            field = 'Diagnosis'
            avg_scores[field] = {}
            avg_samples[field] = {}
            for field in fields:
                avg_scores[field] = {}
                avg_samples[field] = {}
                for patho in pathos:
                    if field in ['Unnecessary Laboratory Tests', 'Unnecessary Imaging']:
                        all_evals[patho] = count_unnecessary(all_evals[patho], field)
                    avg, n = calculate_average(all_evals[patho], field, patho)

                    avg_scores[field][patho] = avg
                    avg_samples[field][patho] = n
            model_scores[model] = avg_scores
        experiment_results[experiment] = model_results
        experiment_evals[experiment] = model_evals
        experiment_scores[experiment] = model_scores
    return experiment_results, experiment_evals, experiment_scores

def check_diagnoses_orig_dr_eval(ids, id_difficulty):
    for patho in ['appendicitis', 'cholecystitis', 'pancreatitis', 'diverticulitis']:
        id_difficulty[patho]['original_dr_eval'] = []
    for i in ids:
        for patho in ['appendicitis', 'cholecystitis', 'pancreatitis', 'diverticulitis']:
            if i in id_difficulty[patho]['first_diag']:
                print(f"{i} is in {patho}")
                id_difficulty[patho]['original_dr_eval'].append(i)
    for patho in "gastritis", "urinary_tract_infection", "hernia", "esophageal_reflux":
        id_difficulty[patho]['original_dr_eval'] = id_difficulty[patho]['dr_eval']
    return id_difficulty

def check_diagnoses(ids, id_difficulty):
    for i in ids:
        for patho in ['appendicitis', 'cholecystitis', 'pancreatitis', 'diverticulitis']:
            if i in id_difficulty[patho]['first_diag']:
                print(f"{i} is in {patho}")

def print_diagnoses(difficulty):
    id_difficulty = pickle.load(open('id_difficulty.pkl', 'rb'))
    diags = {}
    for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']:
        for _id in id_difficulty[patho][difficulty]:
            diags[_id] = patho

    diag_keys = list(diags.keys())
    diag_keys.sort()
    for key in diag_keys:
        print(f"{key} {diags[key]}")

## CDM

### Generate LateX table

In [ ]:
experiment = "CDM_VANILLA"
#models = ["axiong_PMC_LLaMA_13B"]
fields = ['Diagnosis', 'Physical Examination', 'Action Parsing', 'Treatment Parsing', 'Diagnosis Parsing', 'Rounds', 'Invalid Tools', 'Unnecessary Laboratory Tests', 'Unnecessary Imaging']
experiment_results, experiment_evals, experiment_scores = load_scores([experiment], fields=fields)
model_scores = experiment_scores[experiment]

In [ ]:
generate_latex_tables(model_scores, "")

### Plot Diagnostic Accuracy

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

experiment = 'CDM_VANILLA'
fields = [DIAG]
models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ"]

experiment_results, experiment_evals, experiment_scores = load_scores([experiment], fields=fields, models=models)

model_scores = experiment_scores[experiment]

data = []
for model in model_scores.keys():
    mean_diagnosis = np.mean([model_scores[model][DIAG][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']])
    for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']:
        data.append([model, patho.capitalize(), model_scores[model][DIAG][patho]])
    data.append([model, 'Mean', mean_diagnosis])

df = pd.DataFrame(data, columns=['Model', 'Pathology', 'Diagnostic Accuracy'])

# Reshaping the dataframe
#melted_df = df.melt(id_vars=['Model'], var_name='Category', value_name='Diagnostic Accuracy' )

sns.set(style="whitegrid", font_scale=1.4)

# Creating the bar plot
plt.figure(figsize=(12, 4))
df['Model'] = df['Model'].apply(lambda x: prettify_model_name[x])
df['Diagnostic Accuracy'] *= 100
bar_plot = sns.barplot(x='Pathology', y='Diagnostic Accuracy', hue='Model', data=df, palette=color_map)

unique_patho = df['Pathology'].unique()
for i in range(len(unique_patho) - 1):
    bar_plot.axvline(x=i + 0.5, color='gray', linestyle='--', linewidth=1)


# Adding the scores above the bars
for p in bar_plot.patches:
    if p.get_height() > 0:
        bar_plot.annotate(format(p.get_height(), '.1f'), 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha = 'center', va = 'center', 
                      xytext = (0, 9), 
                      textcoords = 'offset points',
                      fontsize=14,)


# Additional plot formatting
plt.title('')
plt.ylabel('Diagnostic Accuracy (%)')
plt.xlabel('')
plt.ylim(0, 100)
plt.legend(bbox_to_anchor=(0.8, 1.18),  ncol=len(model_scores.keys()), frameon=False, fontsize=15)
plt.savefig(f"Figures/DiagnosticAccuraciesCDM.pdf", dpi=300, bbox_inches='tight')
plt.show()


### Plot Imaging Percentage

In [ ]:
import matplotlib.patches as mpatches

from dataset.utils import load_hadm_from_file

experiment = 'CDM_VANILLA'
fields = [DIAG]
models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ"]

experiment_results, experiment_evals, experiment_scores = load_scores([experiment], fields=fields, models=models)

model_scores = experiment_scores[experiment]
model_results = experiment_results[experiment]

modality_map = {
    'CT': 'CT', 
    'Ultrasound': 'Ultrasound', 
    'Radiograph': 'Radiograph',
    'MRI': 'MRI',
    'CTU': 'CT', 
    'ERCP': 'Radiograph', 
    'Upper GI Series': 'Radiograph',
    'MRE': 'MRI', 
    'MRCP': 'MRI',
    'MRA': 'MRI',
    'EUS': 'Ultrasound',
    'Carotid ultrasound': 'Ultrasound',
    'None': 'None', 
    'PTC': 'Other', 
    'HIDA': 'Other', 
    'Drainage': 'Other', 
}

data = []
for model in model_results.keys():
    for patho in model_results[model].keys():
        for _id in model_results[model][patho].keys():
            imaging_requested = False
            for step in model_results[model][patho][_id]['intermediate_steps']:
                if step[0].tool == 'Imaging':
                    tool_input = step[0].tool_input
                    region = tool_input['action_input']["region"]
                    modality = tool_input['action_input']["modality"]
                    if region == "Abdomen":
                        imaging_requested = True
                        modality = modality_map[modality]
                        data.append({"Model": prettify_model_name[model], "Pathology": patho.capitalize(), "Modality": modality})
                        break
            if not imaging_requested:
                data.append({"Model": prettify_model_name[model], "Pathology": patho.capitalize(), "Modality": "None"})

app_hadm_info_firstdiag         = load_hadm_from_file('appendicitis_hadm_info_first_diag')
cholec_hadm_info_firstdiag      = load_hadm_from_file('cholecystitis_hadm_info_first_diag')
pancr_hadm_info_firstdiag       = load_hadm_from_file('pancreatitis_hadm_info_first_diag')
divert_hadm_info_firstdiag      = load_hadm_from_file('diverticulitis_hadm_info_first_diag')

for patho, hadm_info in zip(['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis'], [app_hadm_info_firstdiag, cholec_hadm_info_firstdiag, divert_hadm_info_firstdiag, pancr_hadm_info_firstdiag]):
    for patient in hadm_info.keys():
        for rad in hadm_info[patient]["Radiology"]:
            if rad["Region"] == "Abdomen":
                #data.append({"Model": "Dataset", "Pathology": patho.capitalize(), "Modality": rad["Modality"]})
                data.append({"Model": "MIMIC Doctors", "Pathology": patho.capitalize(), "Modality": modality_map[rad["Modality"]]})
                break

df = pd.DataFrame(data, columns=['Model', 'Pathology', 'Modality'])

sns.set(style="whitegrid", font_scale=1.4)

# 1. Calculating Counts
counts = df.groupby(['Model', 'Pathology', 'Modality']).size().reset_index(name='Counts')

# 2. Calculating Total Counts for Each Pathology and Model
total_counts = counts.groupby(['Model', 'Pathology'])['Counts'].transform('sum')

# 3. Calculating Percentages
counts['Percentage'] = (counts['Counts'] / total_counts) * 100

# Preparing data for grouped bar chart
models = counts['Model'].unique()
pathologies = counts['Pathology'].unique()
modalities = counts['Modality'].unique()

# Number of bars (models) per group (pathology)
n_bars = len(models)

# Width of each bar
bar_width = 0.2

# Define the custom order for modalities
modality_order = ["CT", "Ultrasound", "MRI", "Radiograph", "Other", "None"]

# Defining a color map for modalities
modality_colors = {modality: plt.cm.Accent(i/len(modality_order)) for i, modality in enumerate(modality_order)}

# Defining model order
model_order = ["Llama 2 Chat", "OASST", "WizardLM", "MIMIC Doctors"]

# Define hatch patterns for models
model_hatches = ['/', '.', '-', "x"]
light_edge_colors = ['lightgrey']

# Plotting
plt.figure(figsize=(14, 6))

for j, model in enumerate(model_order):
    offsets = np.arange(len(pathologies)) + (j - n_bars / 2) * bar_width
    bottom_val = np.zeros(len(pathologies))

    for modality in modality_order:
        model_modality_data = counts[(counts['Model'] == model) & (counts['Modality'] == modality)]
        heights = model_modality_data.set_index('Pathology').reindex(pathologies)['Percentage'].fillna(0).tolist()

        # Adding the bars
        bars = plt.bar(offsets, heights, width=bar_width, bottom=bottom_val, color=modality_colors[modality], hatch=model_hatches[j], edgecolor="lightgrey", label=f"{modality}" if j == 0 else "")

        for bar in bars:
            height = bar.get_height()
            if height > 5:  # Only add text labels if the bar segment is visible
                y_position = bar.get_y() + height / 2  # Calculate the y position for the label
                text_color = 'white' if (modality=="Radiograph" or modality=="None") else 'black'  # Choose text color for readability
                plt.text(bar.get_x() + bar.get_width() / 2, y_position, f'{height:.0f}', ha='center', va='center', color=text_color)


        bottom_val += np.array(heights)

plt.xlabel('Pathology')
plt.ylabel('Imaging Modality Requested (%)')
#plt.title('Percentage of Modalities per Pathology by Model')
plt.xticks(np.arange(len(pathologies))-0.1, pathologies)

# Adjusting legend to show modalities only
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(0.9, 1.1),  ncol=len(by_label.keys()), frameon=False, fontsize=16)

# Create custom legend handles for modalities
modality_handles = [mpatches.Patch(color=modality_colors[modality], label=modality) for modality in modality_order]

# Create custom legend handles for models using hatch patterns
dense_model_hatches = ['///', '..', '---', 'xxx']
model_handles = [mpatches.Patch(facecolor='white', edgecolor='black', hatch=dense_model_hatches[j], label=model_order[j]) for j in range(len(models))]

# Combine the modality and model handles
legend_handles = modality_handles + model_handles

# Add the legend to the plot
plt.legend(handles=legend_handles, bbox_to_anchor=(0.95, 1.25), ncol=5, frameon=False, fontsize=15)


plt.savefig(f"Figures/ImagingPercentages.pdf", dpi=300, bbox_inches='tight')

plt.tight_layout()
plt.show()

### Plot Physical Examination Percentage

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

experiment = 'CDM_VANILLA'
fields = ['Physical Examination', 'Late Physical Examination']
models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ"]

experiment_results, experiment_evals, experiment_scores = load_scores([experiment], fields=fields, models=models)

model_scores = experiment_scores[experiment]
model_results = experiment_results[experiment]
model_evals = experiment_evals[experiment]

data = []
for model in model_scores.keys():
    mean_physical = np.mean([model_scores[model]['Physical Examination'][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']])
    mean_physical_late = np.mean([model_scores[model]['Late Physical Examination'][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']])
    data.append([model, mean_physical, mean_physical_late])

df = pd.DataFrame(data, columns=['Model', 'Physical Examination', '(Late) Physical Examination'])
df['Physical Examination'] *= 100
df['(Late) Physical Examination'] *= 100

# Reshaping the dataframe
melted_df = df.melt(id_vars=['Model'], var_name='Category', value_name='Percentage' )

sns.set(style="whitegrid", font_scale=1.4)

# Creating the bar plot
plt.figure(figsize=(10, 4))
melted_df['Model'] = melted_df['Model'].apply(lambda x: prettify_model_name[x])

bar_plot = sns.barplot(x='Category', y='Percentage', hue='Model', data=melted_df, palette=color_map)

# Adding the scores above the bars
for p in bar_plot.patches:
    if p.get_height() > 0:
        bar_plot.annotate(format(p.get_height(), '.1f'), 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha = 'center', va = 'center', 
                      xytext = (0, 9), 
                      textcoords = 'offset points')

# Additional plot formatting
plt.title('')
plt.ylabel('Examination Requested (%)')
plt.xlabel('')
plt.ylim(0, 100)
plt.legend(bbox_to_anchor=(0.9, 1.2),  ncol=len(model_scores.keys()), frameon=False, fontsize=16)
plt.savefig(f"Figures/PhysicalExaminationPercentages.pdf", dpi=300, bbox_inches='tight')
plt.show()


### Plot Lab Test Percentage

In [ ]:
from agents.AgentAction import AgentAction
from dataset.utils import load_hadm_from_file

from evaluators.appendicitis_evaluator import AppendicitisEvaluator
from evaluators.cholecystitis_evaluator import CholecystitisEvaluator
from evaluators.diverticulitis_evaluator import DiverticulitisEvaluator
from evaluators.pancreatitis_evaluator import PancreatitisEvaluator

evaluators = {
    'appendicitis': AppendicitisEvaluator(),
    'cholecystitis': CholecystitisEvaluator(),
    'diverticulitis': DiverticulitisEvaluator(),
    'pancreatitis': PancreatitisEvaluator(),
}

experiment = 'CDM_VANILLA'
fields = []
models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ"]

experiment_results, experiment_evals, experiment_scores = load_scores([experiment], fields=fields, models=models)

model_scores = experiment_scores[experiment]
model_results = experiment_results[experiment]
model_evals = experiment_evals[experiment]

required_lab_tests = {}
for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']:
    required_lab_tests[patho] = evaluators[patho].required_lab_tests

app_hadm_info_firstdiag         = load_hadm_from_file('appendicitis_hadm_info_first_diag')
cholec_hadm_info_firstdiag      = load_hadm_from_file('cholecystitis_hadm_info_first_diag')
pancr_hadm_info_firstdiag       = load_hadm_from_file('pancreatitis_hadm_info_first_diag')
divert_hadm_info_firstdiag      = load_hadm_from_file('diverticulitis_hadm_info_first_diag')

model_evals["MIMIC Doctors"] = {}
for patho, hadm_info in zip(['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis'], [app_hadm_info_firstdiag, cholec_hadm_info_firstdiag, divert_hadm_info_firstdiag, pancr_hadm_info_firstdiag]):
    model_evals["MIMIC Doctors"][patho] = {}
    for patient in hadm_info.keys():
        evaluator = evaluators[patho]
        action = AgentAction(tool = "Laboratory Tests", tool_input = {"action_input" : list(hadm_info[patient]['Laboratory Tests'].keys())}, log = "", custom_parsings=0)
        evaluator.score_laboratory_tests(action)
        eval = {
            "scores": evaluator.scores,
            "answers": evaluator.answers,
        }
        model_evals["MIMIC Doctors"][patho][patient] = eval

data = []
required_lab_test_percentages = {}
for model in model_evals.keys():
    required_lab_test_percentages[model] = {}
    for patho in model_evals[model].keys():
        required_lab_test_percentages[model][patho] = {}
        for required_lab_test in required_lab_tests[patho]:
            required_lab_test_percentages[model][patho][required_lab_test] = 0
        for _id in model_evals[model][patho].keys():
            for required_lab_test in required_lab_tests[patho]:
                required_lab_test_percentages[model][patho][required_lab_test] += 1 if len(model_evals[model][patho][_id]['answers']['Correct Laboratory Tests'][required_lab_test]) else 0
        for required_lab_test in required_lab_tests[patho]:
            required_lab_test_percentages[model][patho][required_lab_test] /= len(model_evals[model][patho].keys()) 
            required_lab_test_percentages[model][patho][required_lab_test] *= 100
            data.append([prettify_model_name[model], patho.capitalize(), required_lab_test, required_lab_test_percentages[model][patho][required_lab_test]])

df = pd.DataFrame(data, columns=['Model', 'Pathology', 'Required Laboratory Test', 'Percentage'])

# Creating the plot
sns.set_theme(style="whitegrid", font_scale=1.6)
fig, axes = plt.subplots(2, 2, figsize=(16, 10), sharey=True)

# Replace seriousness with severity
df['Required Laboratory Test'] = df['Required Laboratory Test'].replace('Seriousness', 'Severity')

# Pathologies and their relevant tests
pathology_tests = {
    'Appendicitis': ['Inflammation'],
    'Cholecystitis': ['Inflammation', 'Liver', 'Gallbladder'],
    'Diverticulitis': ['Inflammation'],
    'Pancreatitis': ['Inflammation', 'Pancreas', 'Severity']
}

# Plotting each pathology in its subplot
for i, (pathology, tests) in enumerate(pathology_tests.items()):
    print(i, pathology)
    ax = axes[i // 2, i % 2]
    pathology_df = df[(df['Pathology'] == pathology) & 
                               (df['Required Laboratory Test'].isin(tests))]
    bar_plot = sns.barplot(x='Required Laboratory Test', y='Percentage', hue='Model', data=pathology_df, ax=ax, palette=color_map)

    unique_patho = pathology_df['Required Laboratory Test'].unique()
    for j in range(len(unique_patho) - 1):
        bar_plot.axvline(x=j + 0.5, color='gray', linestyle='--', linewidth=1)

    ax.set_title(pathology)
    if i > 1:
        ax.set_xlabel('Laboratory Test Category')
    else:
        ax.set_xlabel('')
    ax.set_ylabel('Lab Test Requested (%)')
    bar_plot.get_legend().remove()

    # Print % correct above each bar
    for p in bar_plot.patches:
        if p.get_height() > 0:
            adjust = 0 if p.get_height() < 99 else 10
            color = 'black' if p.get_height() < 99 else 'white'
            _format = '.1f' if p.get_height() < 99 else '.0f'
            bar_plot.annotate(format(p.get_height(), _format), 
                (p.get_x() + p.get_width() / 2., p.get_height() - adjust), 
                ha = 'center', va = 'center', 
                xytext = (0, 9), 
                textcoords = 'offset points',
                fontsize=15, color=color)

# Adjust layout for clarity
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.ylim(0, 100)

plt.legend(bbox_to_anchor=(0.7, 2.57),  ncol=len(model_evals.keys()), frameon=False, fontsize=20)
plt.savefig(f"Figures/LaboratoryTestPercentages.pdf", dpi=300, bbox_inches='tight')

plt.show()

### Plot Instruction Following

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

experiments = ['CDM_VANILLA', 'FI_PLI']
fields = ['Action Parsing', 'Treatment Parsing', 'Diagnosis Parsing', 'Invalid Tools']
models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ"]

experiment_results, experiment_evals, experiment_scores = load_scores(experiments, fields=fields, models=models)

model_scores = experiment_scores["CDM_VANILLA"]
model_results = experiment_results["CDM_VANILLA"]
model_evals = experiment_evals["CDM_VANILLA"]

model_scores_fi = experiment_scores["FI_PLI"]
model_results_fi = experiment_results["FI_PLI"]
model_evals_fi = experiment_evals["FI_PLI"]

data = []
for model in model_scores.keys():
    mean_action_parsing = np.mean([model_scores[model]['Action Parsing'][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']])
    mean_treatment_parsing = np.mean([model_scores[model]['Treatment Parsing'][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']])
    mean_diagnosis_parsing = np.mean([model_scores[model]['Diagnosis Parsing'][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']])
    mean_diagnosis_parsing_fi = np.mean([model_scores_fi[model]['Diagnosis Parsing'][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']])
    mean_invalid_tools = np.mean([model_scores[model]['Invalid Tools'][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']])
    data.append([model, mean_action_parsing, mean_diagnosis_parsing, mean_invalid_tools, mean_diagnosis_parsing_fi])

df = pd.DataFrame(data, columns=['Model', 'Action Parsing', 'Diagnosis Parsing', 'Invalid Tools', 'Diagnosis Parsing FI'])
df["Model"] = df["Model"].apply(lambda x: prettify_model_name[x])
df['Invalid Tools'] = 1 / df['Invalid Tools']
df["Action Parsing"] = 1 / df["Action Parsing"]
#df["Defining Treatment"] = 1 / df["Defining Treatment"]
df["Diagnosis Parsing"] = 1 / df["Diagnosis Parsing"]
df["Diagnosis Parsing FI"] = 1 / df["Diagnosis Parsing FI"]

# Reshaping the dataframe
melted_df = df.melt(id_vars=['Model'], var_name='Category', value_name='Percentage' )

sns.set(style="whitegrid", font_scale=1.4)

# Creating the bar plot
plt.figure(figsize=(12, 4))
bar_plot = sns.barplot(x='Category', y='Percentage', hue='Model', data=melted_df, palette=color_map, order=['Action Parsing', 'Invalid Tools', 'Diagnosis Parsing', 'Diagnosis Parsing FI'])

unique_patho = melted_df['Category'].unique()
for j in range(len(unique_patho) - 1):
    bar_plot.axvline(x=j + 0.5, color='gray', linestyle='--', linewidth=1)

# Adding the scores above the bars
for p in bar_plot.patches:
    if p.get_height() > 0:
        bar_plot.annotate(format(p.get_height(), '.2f'), 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha = 'center', va = 'center', 
                      xytext = (0, 9), 
                      textcoords = 'offset points')

# Additional plot formatting
plt.title('')
plt.ylabel('Average Number of Patients\nUntil Formatting Error')
plt.xlabel('')
plt.ylim(0, 30)
plt.xticks(labels=['MIMIC-CDM\nNext Action Error', 'MIMIC-CDM\nTool Hallucination', 'MIMIC-CDM\nDiagnosis Error', 'MIMIC-CDM-FI\nDiagnosis Error',], ticks=[0, 1, 2, 3])
plt.legend(bbox_to_anchor=(0.85, 1.2),  ncol=len(model_scores.keys()), frameon=False, fontsize=16)
plt.savefig(f"Figures/InstructionFollowingScores.pdf", dpi=300, bbox_inches='tight')
plt.show()


### Plot Treatment Percentage 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator

experiment = 'CDM_VANILLA'
fields = [DIAG]
models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ"]

experiment_results, experiment_evals, experiment_scores = load_scores([experiment], fields=fields, models=models)

model_scores = experiment_scores[experiment]
model_evals = experiment_evals[experiment]

def colo_replace(x):
    if x == "Colonoscopy":
        return "Future\nColonoscopy"
    else:
        return x

# Transform data structure into pandas DataFrame
data = []
treatment_required_counts = {}
for model in model_evals:
    for pathology in model_evals[model]:
        treatment_required_counts[pathology] = {}
        for patient_id in model_evals[model][pathology]:
            treatment_requested = model_evals[model][pathology][patient_id]['answers']['Treatment Requested']
            treatment_required = model_evals[model][pathology][patient_id]['answers']['Treatment Required']
            correctly_diagnosed = model_evals[model][pathology][patient_id]['scores'][DIAG]

            for treatment in treatment_required:
                if treatment_required[treatment]:
                    treatment_required_counts[pathology][treatment] = treatment_required_counts[pathology].get(treatment, 0) + 1
                    if correctly_diagnosed:
                        requested = treatment_requested.get(treatment, False)
                        data.append([model, pathology, treatment, requested]) 

df = pd.DataFrame(data, columns=['Model', 'Pathology', 'Treatment', 'Requested'])
df['Model'] = df['Model'].replace(prettify_model_name)

# Compute percentage of correctly requested treatment
df_agg = df.groupby(['Model', 'Pathology', 'Treatment']).mean().reset_index()
df_agg['Request Correct'] = df_agg['Requested']*100

df_counts = df.groupby(['Pathology', 'Treatment', 'Model']).size().reset_index(name='Counts')

# then merge this df_counts with your df_agg DataFrame
df_agg = pd.merge(df_agg, df_counts,  how='left', left_on=['Pathology','Treatment', 'Model'], right_on = ['Pathology','Treatment', 'Model'])

# Remove entries with count less than 5
# df_agg = df_agg[df_agg['Counts'] >= 5]

unique_pathologies = df_agg['Pathology'].unique()

# Set the style and color palette
sns.set_theme(style="whitegrid", font_scale=1.8)

# Create a 2x2 grid for subplots
fig, axes = plt.subplots(2, 2, figsize=(20, 12), sharey=True)

# Flatten the axes array for easy iteration
axes_flat = axes.flatten()

# Iterate through each pathology and plot on its respective subplot
for i, pathology in enumerate(unique_pathologies):
    if i >= 4:  # Skip if there are more than 4 pathologies
        break
    df_filtered = df_agg[df_agg['Pathology'] == pathology]

    bar_plot = sns.barplot(x='Treatment', y='Request Correct', hue='Model', data=df_filtered, ax=axes_flat[i], palette=color_map)

    unique_patho = df_filtered['Treatment'].unique()
    for j in range(len(unique_patho) - 1):
        bar_plot.axvline(x=j + 0.5, color='gray', linestyle='--', linewidth=1)

    # Print counts below each bar
    for j, bar in enumerate(bar_plot.patches):
        if j < len(df_filtered):
            font_size = 16
            bar_plot.text(bar.get_x() + bar.get_width() / 2, -6, 
                          '{}'.format(df_filtered['Counts'].iloc[j]), 
                          ha='center', va='bottom', fontsize=font_size)
    
    # Print % correct above each bar
    for i,p in enumerate(bar_plot.patches):
        if i < len(df_filtered):
            bar_plot.annotate(format(p.get_height(), '.1f'), 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'center', 
                xytext = (0, 9), 
                textcoords = 'offset points',
                fontsize=16)

    # Set axes labels and titles
    bar_plot.set_xlabel('')
    bar_plot.set_ylabel('Treatment Requested (%)')
    bar_plot.set_title(f'{pathology.capitalize()} Treatment')
    label_with_count = lambda x: f"\n{colo_replace(x)}\n(n={treatment_required_counts[pathology][x]})"
    tick_positions = range(len(df_filtered['Treatment'].unique()))
    bar_plot.xaxis.set_major_locator(FixedLocator(tick_positions))
    bar_plot.set_xticklabels([label_with_count(tick.get_text()) for tick in bar_plot.get_xticklabels()])
    bar_plot.get_legend().remove()

# Hide any unused subplots
for ax in axes_flat[len(unique_pathologies):]:
    ax.axis('off')

# Set global parameters
plt.ylim(0, 105)
plt.setp(axes, xlabel='', ylabel='Treatment Requested (%)')

# Create a single legend
handles, labels = axes_flat[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.03), ncol=3, frameon=False)

# Adjust layout and save the figure
plt.tight_layout()
plt.savefig(f"Figures/TreatmentRequested_Combined.pdf", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Manual check of time component of colonoscopy and cholecystectomy
model_evals = experiment_evals["CDM_VANILLA"]
for model in model_evals.keys():
    print(f"---\n{model}\n---")
    for p in model_evals[model]['diverticulitis']:
        treatment = model_evals[model]['diverticulitis'][p]['answers']['Treatment']
        if model_evals[model]['diverticulitis'][p]['scores']['Gracious Diagnosis'] == 1 and 'colonoscopy' in treatment.lower():
            print(treatment)
            print("--")

In [ ]:
# Manual check of time component of colonoscopy and cholecystectomy
model_evals = experiment_evals["CDM_VANILLA"]
for model in model_evals.keys():
    print(f"---\n{model}\n---")
    for p in model_evals[model]['pancreatitis']:
        treatment = model_evals[model]['pancreatitis'][p]['answers']['Treatment']
        if model_evals[model]['pancreatitis'][p]['scores']['Gracious Diagnosis'] == 1 and 'cholecystectomy' in treatment.lower():
            print(treatment)
            print("---")

### Summarization Effect

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ"]
fields = [DIAG]

experiments = ["CDM_VANILLA", "CDM_NOSUMMARY"]
experiment_results, experiment_evals, experiment_scores = load_scores(experiments, fields=fields, models=models)

pathologies = ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']

data_to_plot = []
for experiment in experiment_scores.keys():
    for model_idx, model_name in enumerate(models):
        for pathology in pathologies:
            data_to_plot.append({
                'Experiment': experiment,
                'Model': model_name,
                'Pathology': pathology,
                'Diagnostic Accuracy': experiment_scores[experiment][model_name][DIAG][pathology],
            })
        # Add mean
        data_to_plot.append({
            'Experiment': experiment,
            'Model': model_name,
            'Pathology': 'Mean',
            'Diagnostic Accuracy': np.mean([experiment_scores[experiment][model_name][DIAG][pathology] for pathology in pathologies]),
        })
df_to_plot = pd.DataFrame(data_to_plot)

sns.set(style="whitegrid", font_scale=1.4)
# Correctly calculating the differences
df_filtered = df_to_plot[df_to_plot["Experiment"].isin(experiments)]

# First, we need to separate the data for each experiment
df_vanilla = df_filtered[df_filtered["Experiment"] == "CDM_VANILLA"]
df_NOSUMMARY = df_filtered[df_filtered["Experiment"] == "CDM_NOSUMMARY"]

# Merging the two datasets on 'Model' and 'Pathology' to calculate differences
df_merged = pd.merge(df_vanilla, df_NOSUMMARY, on=["Model", "Pathology"], suffixes=('_VANILLA', '_NOSUMMARY'))

# Calculating the differences
df_merged['Diagnostic Accuracy'] = df_merged['Diagnostic Accuracy_NOSUMMARY'] - df_merged['Diagnostic Accuracy_VANILLA']

# Selecting relevant columns for plotting
df_differences = df_merged[['Model', 'Pathology', 'Diagnostic Accuracy']]

# Display the first few rows of the dataframe with differences
df_differences.head()

# Capitalizing the pathology names
df_differences['Pathology'] = df_differences['Pathology'].str.capitalize()

df_differences['Model'] = df_differences['Model'].map(prettify_model_name)
df_differences_mean = df_differences[df_differences['Pathology'] == 'Mean']
df_differences_mean.drop(columns=['Pathology'], inplace=True)
df_differences_mean = df_differences_mean.melt(id_vars=['Model'], var_name='Category', value_name='Percentage' )
df_differences_mean['Percentage'] = 100*df_differences_mean['Percentage']

# Setting the font scale
sns.set(style="whitegrid", font_scale=1.4)

# Creating a combined plot
fig, axes = plt.subplots(1, 1, figsize=(12, 6))

barplot = sns.barplot(x='Percentage', y="Model", hue="Category", data=df_differences_mean)
barplot.set_xlabel(f"Change in Mean Diagnostic Accuracy (%)")

# Adjusting layout and showing only one legend
plt.tight_layout()

#plt.legend(bbox_to_anchor=(1.0, 1.15),  ncol=5, frameon=False, fontsize=14)
# remove legend
barplot.get_legend().remove()

plt.title("Effect of Removing Summarization During CDM", fontsize=18)
plt.savefig(f"Figures/SummarizationCDMDelta.pdf", dpi=300, bbox_inches='tight')
plt.show()

## Full Info

### Generate Vanilla Table

In [ ]:
experiment = "FI_PLI"
experiment_results, experiment_evals, experiment_scores = load_scores([experiment])
model_scores = experiment_scores[experiment]

# Replace keys with simplified names
model_scores = {prettify_model_name[name]: model_scores[name] for name in model_scores.keys()}

# Capitalize first letter of each disease
model_scores = {model: {disease.capitalize(): score for disease, score in category_dict['Diagnosis'].items()} for model, category_dict in model_scores.items()}

# Calculate mean performance and add to disease dict
for category, disease_dict in model_scores.items():
    disease_dict["Mean"] = np.mean(list(disease_dict.values()))

In [ ]:
experiment = ""
generate_latex_tables_full_info(model_scores, experiment)

### Plot Diagnostic Accuracy

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

experiments = ['FI_PLI', 'FI_PLI_FEWSHOT']
models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ"]
models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ", "ClinicalCamel-70B-GPTQ", "Meditron-70B-GPTQ"]
fields = [DIAG]

experiment_results, experiment_evals, experiment_scores = load_scores(experiments, models=models, fields=fields)

data = []
for experiment in experiment_scores.keys():
    model_scores = experiment_scores[experiment]
    for model in model_scores.keys():
        mean_diagnosis = np.mean([model_scores[model][DIAG][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']])
        for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']:
            data.append([experiment, model, patho.capitalize(), model_scores[model][DIAG][patho]])
        data.append([experiment, model, 'Mean', mean_diagnosis])

df = pd.DataFrame(data, columns=['Experiment', 'Model', 'Pathology', 'Diagnostic Accuracy'])
df['Diagnostic Accuracy'] *= 100
df['Model'] = df['Model'].apply(lambda x: prettify_model_name[x])

# Extract best experiment of each model based on mean diagnostic accuracy
best_experiments = {}
for model in models:
    model = prettify_model_name[model]
    best_experiments[model] = df[(df['Model'] == model) & (df['Pathology']=='Mean')].sort_values(by=['Diagnostic Accuracy'], ascending=False).iloc[0]['Experiment']

# For each model, remove rows that are not the best experiment
df = df[df.apply(lambda x: x['Experiment'] == best_experiments[x['Model']], axis=1)]

# Reshaping the dataframe
#melted_df = df.melt(id_vars=['Model'], var_name='Category', value_name='Diagnostic Accuracy' )

sns.set(style="whitegrid", font_scale=1.4)

# Creating the bar plot
plt.figure(figsize=(12, 6))
bar_plot = sns.barplot(x='Pathology', y='Diagnostic Accuracy', hue='Model', data=df, palette=color_map)

# Calculate the number of unique pathologies and models
num_pathologies = len(df['Pathology'].unique())
num_models = len(df['Model'].unique())

# Draw vertical dotted lines
for i in range(num_pathologies - 1):
    line_position = (i + 1) - 0.5
    #plt.axvline(x=line_position, color='grey', linestyle='--', linewidth=1)



# Adding the scores above the bars
for p in bar_plot.patches:
    if p.get_height() > 0:
        bar_plot.annotate(format(p.get_height(), '.0f'), 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha = 'center', va = 'center', 
                    xytext = (0, 9), 
                    textcoords = 'offset points',
                    fontsize=14,)


# Additional plot formatting
plt.title('')
plt.ylabel('Diagnostic Accuracy (%)')
plt.xlabel('')
plt.ylim(0, 100)
plt.legend(bbox_to_anchor=(1.0, 1.13),  ncol=len(model_scores.keys()), frameon=False, fontsize=14)
plt.savefig(f"Figures/DiagnosticAccuraciesFI.pdf", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']:
    print(len(experiment_results['FI_PLI']['Llama-2-70B-chat-GPTQ'][patho]))

### Dr Eval

In [ ]:
id_difficulty = pickle.load(open('id_difficulty.pkl', 'rb'))

dr_eval_diags = {}
for patho in id_difficulty.keys():
    for _id in id_difficulty[patho]['dr_eval']:
        dr_eval_diags[_id] = patho

# read csv
df = pd.DataFrame(columns=['Patient IDs', 'Category', 'Doctor', "Correct Diagnosis"])
for dr in ["Knauer", "Huber", "Vielhauer", "Bhagat"]:
    path = f"ClinicalBenchmark/patient_diagnoses_{dr}.csv"
    dr_df = pd.read_csv(path, sep=';')
    dr_df['Doctor'] = dr
    df = pd.concat([df, dr_df])

# Replace Divertikulitis with Diverticulitis
df['Diagnosis'] = df['Category'].replace('Divertikulitis', 'Diverticulitis')
df.drop(columns=['Category'], inplace=True)

# Add correct diagnosis using id_difficulty entry
df['Correct Diagnosis'] = df.apply(lambda row: dr_eval_diags[row['Patient IDs']].capitalize(), axis=1)

accuracies = {}
incorrect_ids = {}
incorrect_diagoses = {}
incorrect_dr = {}
for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']:
    # Calculate accuracy 
    accuracies[patho.capitalize()] = df[df['Correct Diagnosis'] == patho.capitalize()]['Diagnosis'].eq(patho.capitalize()).sum() / len(df[df['Correct Diagnosis'] == patho.capitalize()]['Diagnosis'])

    # Get diagnosis of incorrect predictions
    incorrect = df[df['Correct Diagnosis'] == patho.capitalize()]['Diagnosis'].ne(patho.capitalize())
    incorrect_ids[patho] = list(df[df['Correct Diagnosis'] == patho.capitalize()][incorrect]['Patient IDs'])
    incorrect_diagoses[patho] = list(df[df['Correct Diagnosis'] == patho.capitalize()][incorrect]['Diagnosis'])
    incorrect_dr[patho] = list(df[df['Correct Diagnosis'] == patho.capitalize()][incorrect]['Doctor'])

data_to_plot = []

experiments = ["FI_PLI_FEWSHOT"]
experiments = ["FI_PLI_FEWSHOT", "FI_PLI_FEWSHOT_DR_2024", "FI_PLI_FEWSHOT_DR_2025", "FI_PLI_FEWSHOT_DR_2026", "FI_PLI_FEWSHOT_DR_2027", "FI_PLI_FEWSHOT_DR_2028", "FI_PLI_FEWSHOT_DR_2029", "FI_PLI_FEWSHOT_DR_2030", "FI_PLI_FEWSHOT_DR_2031", "FI_PLI_FEWSHOT_DR_2032"]
models = ["Meditron-70B-GPTQ"]
experiment_results, experiment_evals, experiment_scores = load_scores(experiments, models=models, difficulty="dr_eval", fields=[DIAG])

for experiment in experiments:
    model_scores = {}
    model_scores["Meditron-70B-GPTQ"] = experiment_scores[experiment]["Meditron-70B-GPTQ"]
    # Replace keys with simplified names
    model_scores = {prettify_model_name[name]: model_scores[name] for name in model_scores.keys()}

    # Capitalize first letter of each disease
    model_scores = {model: {disease.capitalize(): score for disease, score in category_dict[DIAG].items()} for model, category_dict in model_scores.items()}

    # Calculate mean performance and add to disease dict
    for category, disease_dict in model_scores.items():
        disease_dict["Mean"] = np.mean(list(disease_dict.values()))

    # Collect data in a list for DataFrame creation
    model_names = model_scores.keys()
    for pathology in ['Appendicitis', 'Cholecystitis', 'Diverticulitis', 'Pancreatitis', 'Mean']:
        for model_idx, model_name in enumerate(model_names):
            score = model_scores[model_name][pathology]
            data_to_plot.append({'Model': model_name, 'Performance': score, 'Pathology': pathology})


experiments = ["FI_PLI"]
experiments = ["FI_PLI", "FI_PLI_DR_2024", "FI_PLI_DR_2025", "FI_PLI_DR_2026", "FI_PLI_DR_2027", "FI_PLI_DR_2028", "FI_PLI_DR_2029", "FI_PLI_DR_2030", "FI_PLI_DR_2031", "FI_PLI_DR_2032",]
models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ", "ClinicalCamel-70B-GPTQ"]
experiment_results, experiment_evals, experiment_scores = load_scores(experiments, models=models, difficulty="dr_eval", fields=[DIAG])

for experiment in experiments:
    model_scores = experiment_scores[experiment]
    # Replace keys with simplified names
    model_scores = {prettify_model_name[name]: model_scores[name] for name in model_scores.keys()}

    # Capitalize first letter of each disease
    model_scores = {model: {disease.capitalize(): score for disease, score in category_dict[DIAG].items()} for model, category_dict in model_scores.items()}

    # Calculate mean performance and add to disease dict
    for category, disease_dict in model_scores.items():
        disease_dict["Mean"] = np.mean(list(disease_dict.values()))

    # Collect data in a list for DataFrame creation
    model_names = model_scores.keys()
    for pathology in ['Appendicitis', 'Cholecystitis', 'Diverticulitis', 'Pancreatitis', 'Mean']:
        for model_idx, model_name in enumerate(model_names):
            score = model_scores[model_name][pathology]
            data_to_plot.append({'Model': model_name, 'Performance': score, 'Pathology': pathology})

# Create the DataFrame
df_to_plot = pd.DataFrame(data_to_plot)

# Multiply performance by 100 to get percentage
df_to_plot['Performance'] = df_to_plot['Performance']*100

# Dr Means
accuracies['Mean'] = np.mean(list(accuracies.values()))
df['Accuracy'] = df['Diagnosis'].eq(df['Correct Diagnosis'])


# Check which patients were incorrectly diagnosed by all doctors
results = {}

for pathology in incorrect_ids:
    id_to_doctors = {}
    for idx, id in enumerate(incorrect_ids[pathology]):
        id_to_doctors.setdefault(id, []).append(incorrect_dr[pathology][idx])
    results[pathology] = id_to_doctors

# Printing the required output
for pathology, id_to_doctors in results.items():
    for id, doctors in id_to_doctors.items():
        output_line = f"{pathology},{id},{','.join(doctors)}"
        print(output_line)

# Calculate mean accuracy for each doctor for each correct diagnosis
dr_stats = df.groupby(['Doctor','Correct Diagnosis']).agg(
    Mean_Accuracy=('Accuracy', 'mean'),
    Std_Err=('Accuracy', 'sem'),
).reset_index()
dr_stats = dr_stats[dr_stats['Correct Diagnosis'].isin(['Appendicitis', 'Cholecystitis', 'Diverticulitis', 'Pancreatitis'])]
dr_stats["Mean_Accuracy"] *= 100
dr_stats["Std_Err"] *= 100

# Add mean of each dr
mean_dr_stats = dr_stats.groupby(['Doctor']).agg(
    Mean_Accuracy=('Mean_Accuracy', 'mean'),
    Std_Err=('Std_Err', 'mean'),
).reset_index()
mean_dr_stats['Correct Diagnosis'] = 'Mean'
dr_stats = pd.concat([dr_stats, mean_dr_stats])
dr_stats['Model'] = 'Doctor'
dr_stats.drop(columns=['Doctor', 'Std_Err'], inplace=True)

import seaborn as sns
import matplotlib.pyplot as plt

dr_stats.rename(columns={'Correct Diagnosis': 'Pathology', 'Mean_Accuracy': 'Performance'}, inplace=True)
dr_stats['Model'] = 'Doctors'
#df_to_plot['Std_Err'] = 0
model_df = pd.concat([df_to_plot, dr_stats])

# Set the style and scale
sns.set(style="whitegrid", font_scale=1.4)

# Create the plot
plt.figure(figsize=(14, 6))
ax = sns.barplot(x="Pathology", y="Performance", hue="Model", data=model_df, palette=color_map, errorbar=("ci", 100), hue_order=["Llama 2 Chat", "OASST", "WizardLM", "Clinical Camel", "Meditron", "Doctors"])

# Calculate the number of unique pathologies and models
num_pathologies = len(model_df['Pathology'].unique())
num_models = len(model_df['Model'].unique())

# Draw vertical dotted lines
for i in range(num_pathologies - 1):
    line_position = (i + 1) - 0.5
    #plt.axvline(x=line_position, color='grey', linestyle='--', linewidth=1)

# Annotate bars with performance
for i, p in enumerate(ax.patches):
        # Doctor should be printed next to the bar because of error bar
        if i > 24:
            if p.get_height() > 0:
                ax.annotate(format(p.get_height(), '.0f'), 
                    (p.get_x() + p.get_width() / 2. + 0.1, p.get_height()), 
                    ha = 'center', va = 'center', 
                    xytext = (0, 9), 
                    textcoords = 'offset points',
                    fontsize=14,
                    )
        else:
            if p.get_height() > 99:
                ax.annotate(format(p.get_height(), '.0f'), 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha = 'center', va = 'center', 
                    xytext = (0, 9), 
                    textcoords = 'offset points',
                    fontsize=12,
                    )
            elif p.get_height() > 0:
                ax.annotate(format(p.get_height(), '.0f'), 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha = 'center', va = 'center', 
                    xytext = (0, 9), 
                    textcoords = 'offset points',
                    fontsize=14,
                    )

ax.set_ylabel("Diagnostic Accuracy (%)")
ax.set_ylim(0, 100)
ax.set_title("", fontsize=30)
ax.set_xlabel("")

# Set legend
models = model_df['Model'].unique()
plt.legend(bbox_to_anchor=(1., 1.18), ncol=len(models), frameon=False, fontsize=15)
plt.tight_layout()

# Show the plot
plt.savefig(f"Figures/LLMsWorseThanDoctors.pdf", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
dr_grouped = df.groupby(['Doctor', 'Correct Diagnosis']).agg(
    Mean_Accuracy=('Accuracy', 'mean'),
    Std_Err=('Accuracy', 'sem'),
).reset_index()
primary = dr_grouped[dr_grouped["Correct Diagnosis"].isin(['Appendicitis', 'Cholecystitis', 'Diverticulitis', 'Pancreatitis'])]

#### Check Normal Distribution

In [ ]:
from scipy import stats

scores_subset = {}
models = list(model_scores.keys())+["Meditron"]

for model in models:
    x = model_df[(model_df['Model'] == model) & (model_df["Pathology"] != "Mean")]['Performance']
    scores_subset[model] = x.values
    print(model)
    print(stats.shapiro(x))
    print()

for dr in ['Knauer', 'Huber', 'Vielhauer', 'Bhagat']:
    x = primary[(primary['Doctor'] == dr)]['Mean_Accuracy']
    scores_subset[dr] = x.values*100
    scores_subset["Doctors"] = scores_subset.get("Doctors", []) + list(x.values*100)
    print(dr)
    print(stats.shapiro(x))
    print()

scores_subset["Doctors"] = np.array(scores_subset["Doctors"])
print("Doctors")
print(stats.shapiro(scores_subset["Doctors"]))

#### Check Equal Variance

In [ ]:
for dr in ['Knauer', 'Huber', 'Vielhauer', 'Bhagat', 'Doctors']:
    for model in models:
        print(dr, model)
        #print(stats.levene(scores_subset[dr], scores_subset[model], center='mean'))
        print(stats.bartlett(scores_subset[dr], scores_subset[model]))
        print()

#### Run T-test Unequal Variances

In [ ]:
for dr in ['Knauer', 'Huber', 'Vielhauer', 'Bhagat', 'Doctors']:
    for model in models:
        print(dr, model)
        ttest_res = stats.ttest_ind(scores_subset[dr], scores_subset[model], equal_var=False)
        print(ttest_res)
        print(ttest_res.pvalue*5)
        print()

#### Generalist vs Specialist

In [ ]:
# Check variances

for specialist_model in ["Clinical Camel", "Meditron"]:
    for generalist_model in ["Llama 2 Chat", "OASST", "WizardLM"]:
        print(specialist_model, generalist_model)
        print(stats.levene(scores_subset[specialist_model], scores_subset[generalist_model], center='mean'))
        print()

In [ ]:
# Run Mann whitney U test
for specialist_model in ["Clinical Camel", "Meditron"]:
    for generalist_model in ["Llama 2 Chat", "OASST", "WizardLM"]:
        print(specialist_model, generalist_model)
        #print(stats.mannwhitneyu(scores_subset[specialist_model], scores_subset[generalist_model], alternative='two-sided'))
        ttest_res = stats.ttest_ind(scores_subset[specialist_model], scores_subset[generalist_model], equal_var=False)
        #print(ttest_res)
        print(ttest_res.pvalue*3)

### Impact No Abbrv

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

experiment = 'FI_PLI_DR_NOABBR'
models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ"]
fields = [DIAG]
#models = ["ClinicalCamel-70B-GPTQ"]

experiment_results, experiment_evals, experiment_scores = load_scores([experiment], models=models, fields=fields, difficulty="original_dr_eval")
model_scores = experiment_scores[experiment]

data = []
for model in model_scores.keys():
    mean_diagnosis = np.mean([model_scores[model][DIAG][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']])
    for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']:
        data.append([model, patho.capitalize(), model_scores[model][DIAG][patho]])
    data.append([model, 'Mean', mean_diagnosis])

df = pd.DataFrame(data, columns=['Model', 'Pathology', 'Diagnostic Accuracy'])
df['Diagnostic Accuracy'] *= 100
df['Model'] = df['Model'].apply(lambda x: prettify_model_name[x])

# Reshaping the dataframe
#melted_df = df.melt(id_vars=['Model'], var_name='Category', value_name='Diagnostic Accuracy' )

sns.set(style="whitegrid", font_scale=1.4)

# Creating the bar plot
plt.figure(figsize=(12, 6))
bar_plot = sns.barplot(x='Pathology', y='Diagnostic Accuracy', hue='Model', data=df, palette=color_map)

# Calculate the number of unique pathologies and models
num_pathologies = len(df['Pathology'].unique())
num_models = len(df['Model'].unique())

# Draw vertical dotted lines
for i in range(num_pathologies - 1):
    line_position = (i + 1) - 0.5
    #plt.axvline(x=line_position, color='grey', linestyle='--', linewidth=1)

# Adding the scores above the bars
for p in bar_plot.patches:
    if p.get_height() > 0:
        bar_plot.annotate(format(p.get_height(), '.1f'), 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha = 'center', va = 'center', 
                      xytext = (0, 9), 
                      textcoords = 'offset points',
                      fontsize=14,)


# Additional plot formatting
plt.title('')
plt.ylabel('Diagnostic Accuracy (%)')
plt.xlabel('')
plt.ylim(0, 100)
plt.legend(bbox_to_anchor=(0.85, 1.15),  ncol=len(model_scores.keys()), frameon=False, fontsize=16)
plt.savefig(f"Figures/DiagnosticAccuraciesFIDrNoAbbrv.pdf", dpi=300, bbox_inches='tight')
plt.show()


### Impact of Prompts

In [ ]:
models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ"]
fields = [DIAG]

prettify_experiment_name = {
    "FI_PLI": "Vanilla Prompt",
    "FI_PLI_NOSYSTEMNOUSER": "No System Or User Instruction",
    "FI_PLI_NOMEDICAL": "No Medical Terminology",
    "FI_PLI_NOSYSTEM": "No System Instruction",
    "FI_PLI_MINIMALSYSTEM": "Minimal System Instruction",
    "FI_PLI_SERIOUS": "Most Serious Final Diagnosis",
    "FI_PLI_MAINDIAGNOSIS": "Main Diagnosis", 
    "FI_PLI_NOFINAL": "Diagnosis",
    "FI_PLI_PRIMARYDIAGNOSIS": "Primary Diagnosis",
}

experiments = ["FI_PLI", "FI_PLI_NOSYSTEMNOUSER", "FI_PLI_NOMEDICAL", "FI_PLI_NOSYSTEM", "FI_PLI_MINIMALSYSTEM", "FI_PLI_NOFINAL", "FI_PLI_MAINDIAGNOSIS", "FI_PLI_PRIMARYDIAGNOSIS", "FI_PLI_SERIOUS"]
experiment_results, experiment_evals, experiment_scores = load_scores(experiments, fields=fields, models=models)

# Save only differences
data_to_plot = []
for experiment in ["FI_PLI_NOSYSTEMNOUSER", "FI_PLI_NOMEDICAL", "FI_PLI_NOSYSTEM", "FI_PLI_MINIMALSYSTEM", "FI_PLI_NOFINAL", "FI_PLI_MAINDIAGNOSIS", "FI_PLI_PRIMARYDIAGNOSIS", "FI_PLI_SERIOUS"]:
    for model in models:
        for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']:
            data_to_plot.append({
                'Experiment': experiment,
                'Model': prettify_model_name[model],
                'Pathology': patho.capitalize(),
                'Diagnostic Accuracy': experiment_scores[experiment][model][DIAG][patho]- experiment_scores["FI_PLI"][model][DIAG][patho],
            })
        # Add mean
        data_to_plot.append({
            'Experiment': experiment,
            'Model': prettify_model_name[model],
            'Pathology': 'Mean',
            'Diagnostic Accuracy': np.mean([experiment_scores[experiment][model][DIAG][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']]) - np.mean([experiment_scores["FI_PLI"][model][DIAG][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']]),
        })
df_differences = pd.DataFrame(data_to_plot)
df_differences['Diagnostic Accuracy'] *= 100

# Setting the font scale
sns.set(style="whitegrid", font_scale=1.8)
plt.figure(figsize=(12, 6))

df_differences['Experiment'] = df_differences['Experiment'].map(prettify_experiment_name)
g = sns.catplot(x='Diagnostic Accuracy', y="Model", hue="Pathology", data=df_differences, kind="bar", palette=color_map, col="Experiment", legend_out=True, col_wrap=4)
for ax in g.axes.flatten():
    ax.set_xlabel(f"Change in Diagnostic\nAccuracy (%)")

    # Get unique models to determine the number of lines needed
    unique_models = df_differences['Model'].unique()
    for i in range(len(unique_models) - 1):
        ax.axhline(y=i + 0.5, color='gray', linestyle='--', linewidth=1)

g.set_titles(col_template="{col_name}", size='large')

# Adjusting layout and showing only one legend
plt.tight_layout()

g._legend.remove()
plt.legend(bbox_to_anchor=(0.5, 2.55),  ncol=5, frameon=False, fontsize=24)
#plt.title("Effect of Including Summarization on Diagnosing with Full Information", fontsize=18)
plt.savefig(f"Figures/PromptFIDelta.pdf", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
df_differences[df_differences["Experiment"]=="Primary Diagnosis"]

### Impact of Only Abnormal Labs

In [ ]:
models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ"]
fields = [DIAG]

experiments = ["FI_PLI", "FI_PLI_ONLYABNORMAL"]
experiment_results, experiment_evals, experiment_scores = load_scores(experiments, fields=fields, models=models)

# Save only differences
data_to_plot = []
for model in models:
    for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']:
        data_to_plot.append({
            'Model': prettify_model_name[model],
            'Pathology': patho.capitalize(),
            'Diagnostic Accuracy': experiment_scores["FI_PLI_ONLYABNORMAL"][model][DIAG][patho] - experiment_scores["FI_PLI"][model][DIAG][patho],
        })
    # Add mean
    data_to_plot.append({
        'Model': prettify_model_name[model],
        'Pathology': 'Mean',
        'Diagnostic Accuracy': np.mean([experiment_scores["FI_PLI_ONLYABNORMAL"][model][DIAG][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']]) - np.mean([experiment_scores["FI_PLI"][model][DIAG][patho] for patho in ['appendicitis', 'cholecystitis', 'diverticulitis', 'pancreatitis']]),
    })
df_differences = pd.DataFrame(data_to_plot)
df_differences['Diagnostic Accuracy'] *= 100

# Setting the font scale
sns.set(style="whitegrid", font_scale=1.4)

# Creating a combined plot
fig, axes = plt.subplots(1, 1, figsize=(15, 7))

barplot = sns.barplot(x='Diagnostic Accuracy', y="Model", hue="Pathology", data=df_differences, palette=color_map)
barplot.set_xlabel(f"Change in Diagnostic Accuracy (%)")

unique_models = df_differences['Model'].unique()
for i in range(len(unique_models) - 1):
    barplot.axhline(y=i + 0.5, color='gray', linestyle='--', linewidth=1)


# Loop through the bars and add text labels
for i, p in enumerate(barplot.patches):
    if i > 14:
        continue
    width = p.get_width() # get bar length
    if width > 0:
        shift = 0.1
    elif width < 0:
        shift = -0.9
    barplot.text(width + shift, # set the text at 0.1 unit right of the bar
                 p.get_y() + p.get_height() / 2, # get Y coordinate + half of the bar's height
                 '{:1.1f}'.format(width), # format the value
                 ha = 'left',   # horizontal alignment
                 va = 'center') # vertical alignment
    

# Adjusting layout and showing only one legend
plt.tight_layout()

plt.legend(bbox_to_anchor=(1, 1.2),  ncol=5, frameon=False, fontsize=18)
plt.title("Effect of Including Only Abnormal Laboratory Results on CDM-FI Diagnostic Accuracy", fontsize=18)
plt.savefig(f"Figures/Abnormal.pdf", dpi=300, bbox_inches='tight')
plt.show()

### Single Modality Diagnosis

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

experiments = ["FI_PLI", "FI_H_PROBS", "FI_P_PROBS", "FI_L_PROBS", "FI_I_PROBS"]
experiment_results, experiment_evals, experiment_scores = load_scores(experiments, fields=[DIAG])
experiment_names_map = {
            "FI_PLI": "All",
            "FI_H_PROBS": "History of Present Illness (HPI)",
            "FI_P_PROBS": "HPI + Physical Examination",
            "FI_L_PROBS": "HPI + Laboratory Tests",
            "FI_I_PROBS": "HPI + Imaging",
        }
experiment_scores = {experiment_names_map[name]: experiment_scores[name] for name in experiment_scores.keys()}
for model in ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", 'WizardLM-70B-V1.0-GPTQ']:
    model_name = prettify_model_name[model]

    # Capitalize first letter of each disease
    experiment_scores = {experiment: {model: {category: {disease.capitalize(): score for disease, score in disease_dict.items()} for category, disease_dict in category_dict.items()} for model, category_dict in exp_dict.items()} for experiment, exp_dict in experiment_scores.items()}

    # Calculate mean performance and add to disease dict
    for experiment, category_dict in experiment_scores.items():
        for category, disease_dict in category_dict[model].items():
            disease_dict["Mean"] = np.mean(list(disease_dict.values()))

    model_scores = {train_type: {k: v for k, v in score_dict[model].items()} for train_type, score_dict in experiment_scores.items()}

# Set the seaborn style and increase font sizes
sns.set(style="whitegrid", font_scale=1.2)

# Set up the plot
model_names = list(experiment_scores['All'].keys())
pretty_model_names = [prettify_model_name[name] for name in model_names]
pathologies = ['Appendicitis', 'Cholecystitis', 'Diverticulitis', 'Pancreatitis', 'Mean']

# Collect data in a list for DataFrame creation
data_to_plot = []
for experiment in experiment_scores.keys():
    model_names = list(experiment_scores[experiment].keys())
    for model_idx, model_name in enumerate(model_names):
        for pathology in pathologies:
            score = experiment_scores[experiment][model_name][DIAG][pathology]
            data_to_plot.append({'Experiment': experiment, 'Model': model_name, 'Performance': score, 'Pathology': pathology})

# Create the DataFrame
df_to_plot = pd.DataFrame(data_to_plot)
df_to_plot['Model'] = df_to_plot['Model'].map(prettify_model_name)
df_to_plot['Performance'] *= 100

# Set up the plot with increased figure size
plt.figure(figsize=(14, 10))


# Plot using Seaborn's boxplot for actual scores
palette = sns.color_palette("muted", n_colors=len(pathologies))  # A Seaborn color scheme
# Filter out the data for the All experiment
df_All = df_to_plot[df_to_plot['Experiment'] == 'All']
df_other_experiments = df_to_plot[df_to_plot['Experiment'] != 'All']

# First, create a catplot with the other experiments
g = sns.catplot(x="Pathology", y="Performance", hue="Experiment", col="Model",
                data=df_other_experiments, kind="strip",
                palette=palette, jitter=False,
                height=5, aspect=1, s=80, legend_out=True)

dt_to_max = pd.concat([df_All, df_other_experiments])
df_grouped_max = dt_to_max.groupby(['Model', 'Pathology'])['Performance'].max().reset_index()
df_grouped_max = df_grouped_max[df_grouped_max['Pathology'] != 'Mean']
df_theoretical_best_performance = df_grouped_max.groupby('Model')['Performance'].mean()
# Now, iterate over each subplot and add a line for All
for ax, model in zip(g.axes.flatten(), df_to_plot['Model'].unique()):
    # Get the All data for this model
    model_All_data = df_All[df_All['Model'] == model]
    # Find the x-axis positions for each category
    x_positions = ax.get_xticks()
    for i, pathology in enumerate(model_All_data['Pathology'].unique()):
        # Find the performance for All for this pathology and model
        performance_ALL = model_All_data[model_All_data['Pathology'] == pathology]['Performance'].item()
        performance_theoretical_best = df_theoretical_best_performance[model].item()
        # Draw the line on the relevant subplot within the category x-axis limits
        ax.hlines(performance_ALL, x_positions[i] - 0.2, x_positions[i] + 0.2, color='black', linestyle='-', lw=2, label='All Information' if i == 0 else "")
        if pathology == 'Mean':
            ax.hlines(performance_theoretical_best, x_positions[i] - 0.2, x_positions[i] + 0.2, color='red', linestyle='-', lw=3, label='Theoretical Best')

        # Find the maximum performance difference for each pathology
        model_data = df_other_experiments[(df_other_experiments['Model'] == model) & (df_other_experiments['Pathology'] == pathology)]
        max_diff = 0
        max_performance = 0
        for j, row in model_data.iterrows():
            difference = row['Performance'] - performance_ALL
            if difference > max_diff:
                max_diff = difference
                max_performance = row['Performance']
        if pathology == 'Mean':
            difference = performance_theoretical_best - performance_ALL
            if difference > max_diff:
                max_diff = difference
                max_performance = performance_theoretical_best

        # Plot arrow and text for the maximum difference
        if max_diff > 0:
            extra_x = 0
            extra_y = 0
            if pathology == "Mean":
                extra_x = -0.2
                extra_y = 4
            if pathology == "Pancreatitis" and model == "Llama 2 Chat":
                extra_y = 4
                extra_x = -0.1
            ax.annotate('', xy=(x_positions[i]+0.2, performance_ALL), xytext=(x_positions[i]+0.2, max_performance+2),
                        arrowprops=dict(color='black', arrowstyle='<|-'), ha='center')
            ax.text(x_positions[i] + 0.25 + extra_x, max_performance + extra_y, f'+{max_diff:.1f}', verticalalignment='center')



for ax in g.axes.flatten():
    # Rotate the x-axis tick labels
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    ax.tick_params(axis='x', labelsize='large')
    ax.tick_params(axis='y', labelsize='large')
    ax.set_xlabel('') 
    ax.set_ylabel('Diagnostic Accuracy (%)', fontsize='x-large')
    ax.set_ylim(0, 100)
g.set_titles(col_template="{col_name}", size='large')


g._legend.remove()
plt.legend(bbox_to_anchor=(0.7, 1.35),  ncol=3, frameon=False, fontsize=16)

plt.savefig(f"Figures/InformationOverload.pdf", dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
df_theoretical_best_performance

### Modality Order

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

experiments = ["FI_PLI", "FI_PIL", "FI_LIP", "FI_LPI", "FI_IPL", "FI_ILP"]
experiment_results, experiment_evals, experiment_scores = load_scores(experiments, fields=[DIAG])

for model in ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", 'WizardLM-70B-V1.0-GPTQ']:
    model_name = prettify_model_name[model]

    # Capitalize first letter of each disease
    experiment_scores = {experiment: {model: {category: {disease.capitalize(): score for disease, score in disease_dict.items()} for category, disease_dict in category_dict.items()} for model, category_dict in exp_dict.items()} for experiment, exp_dict in experiment_scores.items()}

    # Calculate mean performance and add to disease dict
    for experiment, category_dict in experiment_scores.items():
        for category, disease_dict in category_dict[model].items():
            disease_dict["Mean"] = np.mean(list(disease_dict.values()))

    model_scores = {train_type: {k: v for k, v in score_dict[model].items()} for train_type, score_dict in experiment_scores.items()}

    model_names_map = {
        "FI_PLI": "Physical, Lab, Imaging",
        "FI_PIL": "Physical, Imaging, Lab",
        "FI_LIP": "Lab, Imaging, Physical",
        "FI_LPI": "Lab, Physical, Imaging",
        "FI_IPL": "Imaging, Physical, Lab",
        "FI_ILP": "Imaging, Lab, Physical",
    }
    # Replace keys with simplified names
    model_scores = {model_names_map[name]: model_scores[name][DIAG] for name in model_scores.keys()}

    generate_latex_tables_full_info(model_scores, model_name)

# Set the seaborn style and increase font sizes
sns.set(style="whitegrid", font_scale=1.2)

# Set up the plot
model_names = list(experiment_scores['FI_PLI'].keys())
pretty_model_names = [prettify_model_name[name] for name in model_names]
pathologies = ['Appendicitis', 'Cholecystitis', 'Diverticulitis', 'Pancreatitis', 'Mean']

# Collect data in a list for DataFrame creation
data_to_plot = []
for experiment in experiment_scores.keys():
    model_names = list(experiment_scores[experiment].keys())
    for model_idx, model_name in enumerate(model_names):
        for pathology in pathologies:
            score = experiment_scores[experiment][model_name][DIAG][pathology]
            data_to_plot.append({'Experiment': experiment, 'Model': model_name, 'Performance': score, 'Pathology': pathology})

# Create the DataFrame
df_to_plot = pd.DataFrame(data_to_plot)
df_to_plot["Performance"] *= 100

# Calculate range of each model and pathology combination
range_df = (df_to_plot.groupby(['Model', 'Pathology'])['Performance'].max()-df_to_plot.groupby(['Model', 'Pathology'])['Performance'].min()).reset_index()

# Recalc mean
range_df = range_df[range_df['Pathology'] != 'Mean']
range_df_mean = range_df.groupby('Model')['Performance'].mean().reset_index()
range_df_mean['Pathology'] = 'Mean'
range_df = pd.concat([range_df, range_df_mean])

# Set up the plot with increased figure size
plt.figure(figsize=(14, 6))


# Plot using Seaborn's boxplot for actual scores
bar_plot = sns.barplot(x="Model", y="Performance", hue="Pathology", data=range_df, palette=color_map, hue_order=['Appendicitis', 'Cholecystitis', 'Diverticulitis', 'Pancreatitis', 'Mean'])

num_pathologies = len(pathologies)
num_models = len(model_names)

# Draw vertical dotted lines
for i in range(num_models - 1):
    line_position = (i + 1) - 0.5
    #plt.axvline(x=line_position, color='grey', linestyle='--', linewidth=1)

# Adding the scores above the bars
for p in bar_plot.patches:
    if p.get_height() > 0:
        bar_plot.annotate(format(p.get_height(), '.1f'), 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha = 'center', va = 'center', 
                      xytext = (0, 9), 
                      textcoords = 'offset points',
                      fontsize=14,)

# Adjust legend to only show unique pathologies and increase font size
handles, labels = plt.gca().get_legend_handles_labels()
unique_labels = dict(zip(labels, handles))

# Improve plot aesthetics
plt.xticks(range(len(model_names)), pretty_model_names, fontsize='large')
plt.yticks(fontsize='large')
plt.ylim(0, 30)
plt.xlabel('Model', fontsize='x-large')
plt.ylabel('Change in Diagnostic Accuracy (%)', fontsize='large')
plt.title('Changing the Order of Information Changes Diagnostic Accuracy', fontsize='x-large', y=1.03)
plt.legend(bbox_to_anchor=(1.0, 1.3),  ncol=len(pathologies), frameon=False, fontsize='large')

# Ensure everything fits without overlapping
plt.tight_layout()

# SAVE
plt.savefig(f"Figures/SensitiveToOrder.pdf", dpi=300, bbox_inches='tight')

# Display the plot
plt.show()


In [ ]:
range_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

experiments = ["FI_PLI", "FI_PIL", "FI_LIP", "FI_LPI", "FI_IPL", "FI_ILP"]
experiment_results, experiment_evals, experiment_scores = load_scores(experiments, fields=[DIAG])

for model in ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", 'WizardLM-70B-V1.0-GPTQ']:
    model_name = prettify_model_name[model]

    # Capitalize first letter of each disease
    experiment_scores = {experiment: {model: {category: {disease.capitalize(): score for disease, score in disease_dict.items()} for category, disease_dict in category_dict.items()} for model, category_dict in exp_dict.items()} for experiment, exp_dict in experiment_scores.items()}

    # Calculate mean performance and add to disease dict
    for experiment, category_dict in experiment_scores.items():
        for category, disease_dict in category_dict[model].items():
            disease_dict["Mean"] = np.mean(list(disease_dict.values()))

    model_scores = {train_type: {k: v for k, v in score_dict[model].items()} for train_type, score_dict in experiment_scores.items()}

    model_names_map = {
        "FI_PLI": "Physical, Lab, Imaging",
        "FI_PIL": "Physical, Imaging, Lab",
        "FI_LIP": "Lab, Imaging, Physical",
        "FI_LPI": "Lab, Physical, Imaging",
        "FI_IPL": "Imaging, Physical, Lab",
        "FI_ILP": "Imaging, Lab, Physical",
    }
    # Replace keys with simplified names
    model_scores = {model_names_map[name]: model_scores[name][DIAG] for name in model_scores.keys()}

    #generate_latex_tables_full_info(model_scores, model_name)

# Set the seaborn style and increase font sizes
sns.set(style="whitegrid", font_scale=1.2)

# Set up the plot
model_names = list(experiment_scores['FI_PLI'].keys())
pretty_model_names = [prettify_model_name[name] for name in model_names]
pathologies = ['Appendicitis', 'Cholecystitis', 'Diverticulitis', 'Pancreatitis', 'Mean']

# Collect data in a list for DataFrame creation
data_to_plot = []
for experiment in experiment_scores.keys():
    model_names = list(experiment_scores[experiment].keys())
    for model_idx, model_name in enumerate(model_names):
        for pathology in pathologies:
            score = experiment_scores[experiment][model_name][DIAG][pathology]
            data_to_plot.append({'Experiment': experiment, 'Model': model_name, 'Performance': score, 'Pathology': pathology})

# Create the DataFrame
df_to_plot = pd.DataFrame(data_to_plot)
df_to_plot["Performance"] *= 100

# Set up the plot with increased figure size
plt.figure(figsize=(14, 6))


# Plot using Seaborn's boxplot for actual scores
boxplot = sns.boxplot(x="Model", y="Performance", hue="Pathology", data=df_to_plot, palette=color_map)
ax = plt.gca()

num_pathologies = len(pathologies)
num_models = len(model_names)
num_patches_per_box = 6

for i in range(0, len(ax.lines), num_patches_per_box):
    box = ax.patches[i // num_patches_per_box]
    # Get the data for this particular box
    pathology_index = (i // num_patches_per_box) // num_models
    model_index = (i // num_patches_per_box) % num_models
    model_name = model_names[model_index]
    pathology = pathologies[pathology_index]
    group_data = df_to_plot[(df_to_plot['Model'] == model_name) & (df_to_plot['Pathology'] == pathology)]['Performance']

    # Calculate the max and min of the group
    max_value = group_data.max()
    min_value = group_data.min()
    whisker_diff = max_value - min_value
    
    # Calculate the location to place the text: above the upper whisker
    whisker = ax.lines[i+1]  # upper whisker line
    x, y = (whisker.get_xdata()[0], whisker.get_ydata()[1])
    
    # Add text annotation above each whisker with the whisker difference
    ax.text(x, y + (0.02 * (ax.get_ylim()[1] - ax.get_ylim()[0])), f'{whisker_diff:.2f}',
             horizontalalignment='center', size='small', color='black', weight='semibold')


# Adjust legend to only show unique pathologies and increase font size
handles, labels = plt.gca().get_legend_handles_labels()
unique_labels = dict(zip(labels, handles))

# Add vertical dashed lines in between the models
for i in range(len(model_names) - 1):
    plt.axvline(x=i + 0.5, color='grey', linestyle='--', linewidth=1)

# Improve plot aesthetics
plt.xticks(range(len(model_names)), pretty_model_names, fontsize='large')
plt.yticks(fontsize='large')
plt.ylim(0, 100)
plt.xlabel('Model', fontsize='x-large')
plt.ylabel('Diagnostic Accuracy (%)', fontsize='x-large')
plt.title('Changing the Order of Information Changes Diagnostic Accuracy', fontsize='x-large', y=1.03)
plt.legend(bbox_to_anchor=(1.0, 1.3),  ncol=len(pathologies), frameon=False, fontsize='large')

# Ensure everything fits without overlapping
plt.tight_layout()

# SAVE
plt.savefig(f"Figures/SensitiveToOrder_boxplot.pdf", dpi=300, bbox_inches='tight')

# Display the plot
plt.show()


In [ ]:
import pandas as pd

# Assuming df_to_plot is your existing DataFrame
models = df_to_plot['Model'].unique()

experiment_mapping = {
    "FI_PLI": "Physical, Lab, Imaging",
    "FI_PIL": "Physical, Imaging, Lab",
    "FI_LIP": "Lab, Imaging, Physical",
    "FI_LPI": "Lab, Physical, Imaging",
    "FI_IPL": "Imaging, Physical, Lab",
    "FI_ILP": "Imaging, Lab, Physical",
}

# Function to add newline in column names after the second comma
def format_column_name(name):
    parts = name.split(', ')
    if len(parts) > 2:
        return '\\thead{' + ', '.join(parts[:2]) + '\\\\' + ', '.join(parts[2:]) + '}'
    else:
        return '\\thead{' + name + '}'

latex_tables = {}
for model in models:
    # Pivot the DataFrame
    pivoted_df = df_to_plot[df_to_plot['Model'] == model].pivot(index='Pathology', columns='Experiment', values='Performance')
    
    # Rename and format columns
    formatted_columns = {col: format_column_name(experiment_mapping[col]) for col in pivoted_df.columns}
    pivoted_df.rename(columns=formatted_columns, inplace=True)

    # Move row 'Mean' to the end
    pivoted_df = pivoted_df.reindex(['Appendicitis', 'Cholecystitis', 'Diverticulitis', 'Pancreatitis', 'Mean'])

    # Bold the max value in each row
    for row in pivoted_df.index:
        max_value = pivoted_df.loc[row].max()
        pivoted_df.loc[row] = pivoted_df.loc[row].apply(lambda x: f"\\textbf{{{x:.2f}}}" if x == max_value else f"{x:.2f}")

    # Export to LaTeX
    latex_tables[model] = pivoted_df.to_latex(escape=False, column_format='ccccccc')

# latex_tables will contain the LaTeX strings for each model
for model in models:
    print(latex_tables[model])


### Lab Test Eval

In [ ]:
experiments = ["RRTEST_FEWSHOT", "RRTEST"]
for experiment in experiments:
    models = ["Llama-2-70B-chat-GPTQ", "Llama2-70B-OASST-SFT-v10-GPTQ", "WizardLM-70B-V1.0-GPTQ"]
    data_to_plot = []
    for model in models:
        results_path = f"logs/SOTA/{experiment}/{model}_*_{experiment}_results.pkl"
        results = []
        for r in read_from_pickle_file(glob.glob(results_path)[0]):
            results.append(r)

        low_preds = []
        normal_preds = []
        high_preds = []
        for r in results:
            for res in r:
                pred = res['Result']
                gt = res['GT']
                if gt == 'Low':
                    low_preds.append(pred)
                elif gt == 'Normal':
                    normal_preds.append(pred)
                elif gt == 'High':
                    high_preds.append(pred)

        acc_low = sum([1 if pred == 'Low' else 0 for pred in low_preds]) / len(low_preds)
        acc_normal = sum([1 if pred == 'Normal' else 0 for pred in normal_preds]) / len(normal_preds)
        acc_high = sum([1 if pred == 'High' else 0 for pred in high_preds]) / len(high_preds)

        data_to_plot.append({
            'Model': model,
            'Lab Test Result': 'Low',
            'Accuracy': acc_low,
        })
        data_to_plot.append({
            'Model': model,
            'Lab Test Result': 'Normal',
            'Accuracy': acc_normal,
        })
        data_to_plot.append({
            'Model': model,
            'Lab Test Result': 'High',
            'Accuracy': acc_high,
        })
    df_to_plot = pd.DataFrame(data_to_plot)

    sns.set(style="whitegrid", font_scale=1.4)

    # Creating the bar plot
    plt.figure(figsize=(12, 6))
    df_to_plot['Model'] = df_to_plot['Model'].map(prettify_model_name)
    df_to_plot['Accuracy'] *= 100
    bar_plot = sns.barplot(x='Lab Test Result', y='Accuracy', hue='Model', data=df_to_plot, palette=color_map)

    # Adjust xtick fontsize
    plt.xticks(fontsize=18)

    # Adding the scores above the bars
    for p in bar_plot.patches:
        if p.get_height() > 0:
            bar_plot.annotate(format(p.get_height(), '.2f'), 
                        (p.get_x() + p.get_width() / 2., p.get_height()), 
                        ha = 'center', va = 'center', 
                        xytext = (0, 9), 
                        textcoords = 'offset points',
                        fontsize=16,)

    # Calculate counts for each 'Lab Test Result'
    counts = {
        "Low": len(low_preds),
        "Normal": len(normal_preds),
        "High": len(high_preds),
    }
    # Annotate counts for each 'Lab Test Result'
    for index, count in counts.items():
        plt.text(x=index, y=-0.10, s=f'n = {count}', ha='center', va='center', fontsize=18, transform=bar_plot.get_xaxis_transform())


    # Additional plot formatting
    plt.title('')
    plt.ylabel('Accuracy (%)', fontsize=22)
    plt.xlabel('')
    plt.ylim(0, 100)
    plt.legend(bbox_to_anchor=(0.9, 1.15),  ncol=len(model_scores.keys()), frameon=False, fontsize=18)
    plt.savefig(f"Figures/LabTest{experiment}.pdf", dpi=300, bbox_inches='tight')
    plt.show()